In [29]:
# Creamos las funciones solicitadas
# def PlayTimeGenre( genero : str ): #Debe devolver año con mas horas jugadas para dicho género.
# Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}
# Modificaciones a los dataframes para mejorar el rendimiento de las funciones

import pandas as pd

df_reviews = pd.read_csv("reviews_eda.csv")
df_items = pd.read_csv("items_eda.csv")
df_games = pd.read_csv("games_eda.csv")

playtime_item = df_items.groupby('item_id')['playtime_forever'].sum().reset_index()
df_merge = pd.merge(df_games, playtime_item, on='item_id', how='inner')
df_PlayTimeGenre = df_merge[["genres","year","playtime_forever"]]
df_PlayTimeGenre =df_PlayTimeGenre.dropna()
df_PlayTimeGenre.to_csv(r'df_PlayTimeGenre.csv',index=False)

In [30]:
# Filtramos el DataFrame para obtener solo las filas que contienen el género específico
# Buscamos el año con la mayor suma de 'playtime_forever'

def PlayTimeGenre(genero : str):
    
    filtro_genero = df_PlayTimeGenre['genres'].str.contains(genero, case=False, na=False)
    df_filtrado = df_PlayTimeGenre[filtro_genero]
    df_agrupado = df_filtrado.groupby('year')['playtime_forever'].sum().reset_index()
    df_agrupado['year'] = df_agrupado['year'].astype(int)
    anio_mayor_suma = df_agrupado.loc[df_agrupado['playtime_forever'].idxmax(), 'year']
    respuesta = {f"Año de lanzamiento con más horas jugadas para Género el {genero}": anio_mayor_suma}
    return respuesta 

PlayTimeGenre("action")

{'Año de lanzamiento con más horas jugadas para Género el action': 2012}

In [31]:
# Fitramos el DataFrame, eliminamos outliers
# Calcular el rango intercuartílico (IQR) y definimos los límites superior e inferior para identificar outliers
# Filtrar el DataFrame para mantener solo los valores dentro de los límites

df_aux = df_games[["item_id","genres","year"]]
df_UserForGenre = pd.merge(df_items,df_aux, on = "item_id")
df_UserForGenre =df_UserForGenre[["user_id","genres","year","playtime_forever"]]
def eliminar_outliers_iqr(df, columna):
    
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
   
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
   
    df_filtrado = df[(df[columna] >= limite_inferior) & (df[columna] <= limite_superior)]

    return df_filtrado

df_UserForGenre = eliminar_outliers_iqr(df_UserForGenre, 'playtime_forever')
df_UserForGenre.to_csv(r'df_UserForGenre.csv',index=False)

In [32]:
# Filtramos el DataFrame para el género especificado
# Agrupamos por 'user_id' y 'year', sumamamos las horas jugadas
# Buscamos el usuario con la máxima suma de horas jugadas
# Filtramos el DataFrame para el usuario con máxima suma de horas jugadas


def userforgenre(genero: str):
    
    f_genero = df_UserForGenre['genres'].str.contains(genero, case=False, na=False)
    df_filt = df_UserForGenre[f_genero]
    
    df_sum = df_filt.groupby(['user_id', 'year'])['playtime_forever'].sum().reset_index()

    max_playtime = df_sum['playtime_forever'].idxmax()
    user_sum_playtime = df_sum.loc[max_playtime, 'user_id']

    df_user = df_sum[df_sum['user_id'] == user_sum_playtime]

    res = [{'Año': int(row['year']), 'Horas': int(row['playtime_forever'])} for _, row in df_user.iterrows()]
    
    return {"Usuario con más horas jugadas para Género {}:".format(genero): user_sum_playtime, "Horas jugadas": res}


userforgenre('actIon')

{'Usuario con más horas jugadas para Género actIon:': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 1998, 'Horas': 2},
  {'Año': 1999, 'Horas': 225},
  {'Año': 2001, 'Horas': 11},
  {'Año': 2002, 'Horas': 1},
  {'Año': 2004, 'Horas': 225},
  {'Año': 2005, 'Horas': 1117},
  {'Año': 2006, 'Horas': 138},
  {'Año': 2007, 'Horas': 684},
  {'Año': 2008, 'Horas': 1871},
  {'Año': 2009, 'Horas': 4065},
  {'Año': 2010, 'Horas': 5720},
  {'Año': 2011, 'Horas': 12612},
  {'Año': 2012, 'Horas': 19514},
  {'Año': 2013, 'Horas': 20546},
  {'Año': 2014, 'Horas': 48114},
  {'Año': 2015, 'Horas': 57145},
  {'Año': 2016, 'Horas': 38991},
  {'Año': 2017, 'Horas': 3716}]}

In [33]:
# Optimizamos el dataFrame para depuracion de datos de la funcion

df_reviews["posted_date"] = pd.to_datetime(df_reviews["posted_date"])
df_reviews["year_posted"] = df_reviews["posted_date"].dt.year
df_UserRecommend = df_reviews[['item_id',"recommend",'year_posted']]
df_aux= df_games[["item_id","title"]]
df_UserRecommend = pd.merge(df_UserRecommend,df_aux, on="item_id", how="inner")
df_UserRecommend = df_UserRecommend[['title','year_posted',"recommend"]]
df_UserRecommend.to_csv(r'df_UserRecommend.csv',index=False)

In [34]:
# Contar el número de recomendaciones para cada juego
# Crear el formato de salida

def UsersRecommend(año: int):
    
    df = df_UserRecommend[(df_UserRecommend['year_posted'] == año) & (df_UserRecommend['recommend'] == True)]
    top_juegos = df['title'].value_counts().head(3)
    final_res = [{"Puesto {}: {}".format(i+1, juego): recomendaciones} for i, (juego, recomendaciones) in enumerate(top_juegos.items())]

    return final_res


UsersRecommend(2012)

[]

In [35]:
# Optimizamos el dataFrame para depuracion de datos de la funcion

df_reviews.head(5)
df_NotRecommend = df_reviews[["item_id","recommend","year_posted"]]
df_info = df_games[["item_id","developer"]]
df_NotRecommend = pd.merge(df_NotRecommend,df_info, on ="item_id",how= "inner")
df_NotRecommend = df_NotRecommend[(df_NotRecommend['recommend'] == False)]
df_NotRecommend = df_NotRecommend[["developer", "year_posted"]]
df_NotRecommend.to_csv(r'df_NotRecommend.csv',index=False)

In [36]:
# Filtramos el DataFrame por el año especificado
# Contamos la cantidad de publicaciones por desarrollador
# Tomamos los primeros N resultados
# Creamos la lista con el formato solicitado


def UserNotRecommend(year):

    df_filtered = df_NotRecommend[df_NotRecommend['year_posted'] == year]
    developer_counts = df_filtered['developer'].value_counts()
    top_developers = developer_counts.head(3)
    result_list = [{"Puesto {}: {}".format(i + 1, developer): count} for i, (developer, count) in enumerate(top_developers.items())]

    return result_list
UserNotRecommend(2013)

[]

In [37]:
df_reviews.head()

,user_id,item_id,helpful,recommend,review,posted_date,sentimiento,sentiment_analysis,year_posted
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,1970-01-01 00:00:00.000002011,0.8481,2,1970
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,1970-01-01 00:00:00.000002011,0.2263,2,1970
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,1970-01-01 00:00:00.000002011,0.9117,2,1970
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,1970-01-01 00:00:00.000002014,0.9566,2,1970
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,1970-01-01 00:00:00.000002013,0.9708,2,1970


In [38]:
# Optimizamos el dataFrame para depuracion de datos de la funcion


df_10 = df_reviews[["item_id","sentiment_analysis"]]
df_11 = df_games[["year","item_id"]]
df_Sentiment_Analysis = pd.merge(df_10,df_11, on = "item_id", how= "inner") 
df_Sentiment_Analysis = df_Sentiment_Analysis.groupby(['year', 'sentiment_analysis']).size().unstack(fill_value=0)
df_Sentiment_Analysis = df_Sentiment_Analysis.reset_index(drop=False)
nuevos_nombres = ["Año",'Negativo', 'Neutro',"Positivo"]
df_Sentiment_Analysis.columns = nuevos_nombres
df_Sentiment_Analysis["Año"] = df_Sentiment_Analysis["Año"].astype(str).str.lower()
df_Sentiment_Analysis.to_csv("df_Sentiment_Analysis.csv", index=False )

In [39]:
df_Sentiment_Analysis.head()

,Año,Negativo,Neutro,Positivo
0,1989,0,0,1
1,1990,1,0,4
2,1991,0,0,1
3,1992,1,0,2
4,1993,1,1,2


In [40]:
# Filtra el DataFrame para obtener la fila correspondiente al desarrollador
# Extrae los valores de las columnas Negativo, Neutro y Positivo
# Crea un diccionario con la información en el formato solicitado

def Sentiment_analysis(desarrollador):
    
    desarrollador =desarrollador.lower()
    desarrollador_fila = df_Sentiment_Analysis[df_Sentiment_Analysis['Año'] == desarrollador]

    
    negativo = desarrollador_fila['Negativo'].values[0]
    neutro = desarrollador_fila['Neutro'].values[0]
    positivo = desarrollador_fila['Positivo'].values[0]

    
    resultado = {
        desarrollador: {
            'Negative': negativo,
            'Neutral': neutro,
            'Positive': positivo
        }
    }

    return resultado


In [41]:
Sentiment_analysis('2013')

{'2013': {'Negative': 952, 'Neutral': 1502, 'Positive': 4172}}